![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
'''
Author: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
Date: 2025-11-18 14:37:31
LastEditors: error: error: git config user.name & please set dead value or install git && error: git config user.email & please set dead value or install git & please set dead value or install git
LastEditTime: 2025-11-22 23:32:10
FilePath: \01_data_download\02_data_download_run.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
# === 第1步：筛选符合正则的文件（支持 max_depth） ===
import os
import re
from pathlib import Path
import json
def find_files(base_dir=".", pattern=None, max_depth=None):
    """
    遍历目录并筛选出符合正则的文件
    :param base_dir: 要遍历的根目录
    :param pattern: 文件名匹配正则表达式
    :param max_depth: 最大遍历深度（None 表示不限制）
    """
    base_path = Path(base_dir).resolve()
    regex = re.compile(pattern) if pattern else None
    matched_files = []

    for root, dirs, files in os.walk(base_path):
        # 计算当前深度
        depth = len(Path(root).relative_to(base_path).parts)
        if max_depth is not None and depth > max_depth:
            dirs[:] = []  # 不再深入
            continue

        for f in files:
            fpath = Path(root) / f
            if regex is None or regex.search(f):
                matched_files.append(fpath)

    print(f"在目录 {base_path} 中找到 {len(matched_files)} 个匹配文件：")
    matched_files = sorted(matched_files, key=lambda x: str(x).lower())

    for f in matched_files:
        print("   ", f)
    print("-" * 60)
    return matched_files


# === 第2步：并行生成 Base64 下载链接 ===
import base64
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display, HTML

def generate_base64_link(file_path):
    """
    读取单个文件并生成 Base64 下载链接
    """
    file_name = os.path.basename(file_path)
    with open(file_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode()
    return f'<a download="{file_path}" href="data:application/zip;base64,{encoded}">下载 {file_name}</a>'

def create_base64_links_parallel(files, max_workers=8):
    """
    并行生成 Base64 下载链接并显示
    """
    if not files:
        return

    print(f" 开始并行生成 {len(files)} 个文件的 Base64 链接...")
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(generate_base64_link, f): f for f in files}

        for future in as_completed(futures):
            file_path = futures[future]
            try:
                link_html = future.result()
                display(HTML(link_html))
            except Exception as e:
                print(f"  生成 {file_path} 链接失败: {e}")

    print(f" 全部完成，共生成 {len(files)} 个链接。")

def split_list(lst, chunk_size):
    """将列表按指定大小分块"""
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

file_list=[]
#file_list += find_files("/Data/alternative/interest-rate", pattern=rf".*", max_depth=1) # 可选
#file_list += find_files("/Data/future_old/comex/factor_files", pattern=r".*20251114.zip", max_depth=1)  # 下载一个即可，解压
#file_list += find_files("/Data/future_old/cme/factor_files", pattern=r".*20251114.zip", max_depth=1)  # 下载一个即可，解压
#file_list += find_files("/Data/future_old/comex/map_files", pattern=r".*20251114.zip", max_depth=1) # 下载一个即可，解压
#file_list += find_files("/Data/future_old/cme/map_files", pattern=r".*20251114.zip", max_depth=1)  # 下载一个即可，解压

# file_list += find_files("/Data/future/comex/daily", pattern=r"(^gc_)|(^si_).*", max_depth=1)
#file_list += find_files("/Data/future_old/cme/daily", pattern=r"(^nq_)|(^es_).*", max_depth=1) #! 云端future 有2个，一个是 future_old符合文档中的命名标准，future不符合，应该会在后续版本更新


year = 201710
file_list += find_files("/Data/future/comex/minute/gc", pattern=rf"^{year}.*t.zip", max_depth=1)

batches = list(split_list(file_list, 10))

print(f"共计 {len(file_list)} 个匹配文件")

In [2]:
create_base64_links_parallel(batches[0], max_workers=2)

In [3]:
create_base64_links_parallel(batches[1], max_workers=2)

In [4]:
create_base64_links_parallel(batches[2], max_workers=2)

In [5]:
create_base64_links_parallel(batches[3], max_workers=2)

In [ ]:
create_base64_links_parallel(batches[4], max_workers=2)

In [ ]:
create_base64_links_parallel(batches[5], max_workers=2)

In [ ]:
create_base64_links_parallel(batches[6], max_workers=2)